In [1]:
import seaborn as sns 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from scipy.stats.mstats import winsorize
# %matplotlib inline

#윈도우 폰트 
from matplotlib import rc
rc('font', family='Malgun Gothic')

plt.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')

## 확장기 MinMax  

In [12]:
select_min_final =['RETA','매출액영업이익률','총자본영업이익률','매출액순이익률',
             '이자보상배율','차입금의존도','vol매출액영업이익률','vol매출액총이익률']

expansion_min = pd.read_csv('../Data/WIN_UP/expansion_win_min_smote.csv',encoding='cp949')
test_min = pd.read_csv('../Data/WIN_UP/expansion_win_min_test.csv',encoding='cp949')

# train
X_train_min = expansion_min[select_min_final]
y_train_min = expansion_min['부도']

# test
X_test = test_min[select_min_final]
y_test = test_min.iloc[:,-1]

In [15]:
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, accuracy_score, f1_score
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression,  LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance
# from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from tqdm.notebook import tqdm
from sklearn.preprocessing import Binarizer
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import GridSearchCV

# df = pd.DataFrame(columns =['models','Accuracy','Precision', 'Recall', 'F1 score','Auc_Score'])



class Model_Optimization():

      def __init__(self):
            
            global model_df
            model_df = pd.DataFrame(columns =['thresholds','models','Accuracy','Precision', 'Recall', 'F1 score','Auc_Score']) 


      #최적 파라미터 찾는 함수
      def grid(self,model_name, X_train=X_train_min, y_train=y_train_min, X_test=X_test,y_test=y_test):
            global model_df

            self.model_name = model_name
            self.X_test = X_test
            self.y_test = y_test
            self.X_train = X_train
            self.y_train  = y_train

            if model_name == "DecisionTreeClassifier":
                  dt = DecisionTreeClassifier()
                  parameters = [{"max_depth": [3,5,7], "min_samples_split":[3,5,7]}]
                  
            elif model_name == "RandomForestClassifier":
                  dt = RandomForestClassifier()
                  parameters = [{"max_depth": [3,5,7], "min_samples_split":[3,5,7]}]
                  
            elif model_name == "LogisticRegression":
                  dt = LogisticRegression()
                  parameters = {'penalty':['l2', 'l1'],
                  'C':[0.01, 0.1, 1, 5, 10]}
                  
            elif model_name == "Linear SVM": 
                  dt = SVC(probability=True)
                  parameters = [{'kernel':['linear'], 'C':[0.001, 0.01, 0.025, 0.1, 1, 10, 100]}]
                  
            elif model_name == "RBF SVM": 
                  dt = SVC(probability=True)
                  parameters = [{'gamma':[0.001, 0.01, 0.1, 1, 10, 100],'kernel':['rbf'],'C':[0.001, 0.01, 0.025, 0.1, 1, 10, 100]}]

            elif model_name == "AdaBoostClassifier":
                  dt= AdaBoostClassifier(random_state=0)
                  parameters={'n_estimators':[5]}
                  
            elif model_name == "GradientBoostingClassifier":
                  dt=GradientBoostingClassifier(random_state=0)
                  parameters = {
                  'n_estimators' : [100, 500],
                  'learning_rate' : [0.05, 0.1]
                              }

            elif model_name == "XGBClassifier":
                  dt=XGBClassifier(random_state=0,verbose=1)
                  parameters={'n_estimators':[100, 500], 'learning_rate':[0.05, 0.1], 'max_depth':[3,4]}
                  
            # elif model_name == "LGBMClassifier":
            #       dt=LGBMClassifier(random_state=0)
            #       parameters={'n_estimators':[400,800], 'learning_rate':[0.05, 0.1] , 'max_depth':[3,4]}
            
            elif model_name == "KNeighborsClassifier":
                  dt = KNeighborsClassifier()
                  parameters = {'n_neighbors':[3]}
                  
            elif model_name == 'MLPClassifier':
                  dt = MLPClassifier(random_state=0)
                  parameters={'max_iter':[1000], 'hidden_layer_sizes':[1], 'activation':['logistic'],
                              'solver':['sgd'], 'alpha':[0.01], 'batch_size':[32],
                              'learning_rate_init':[0.1], 'max_iter':[500]}
                  
            elif model_name == 'GaussianProcessClassifier':
                  dt = GaussianProcessClassifier(random_state=0)
                  parameters={'kernel': [1.0*RBF(1.0)]}
                  
            elif model_name == 'GaussianNB':
                  dt = GaussianNB()
                  parameters={}
                  
            elif model_name =='QuadraticDiscriminantAnalysis':
                  #선형판별분석
                  dt = QuadraticDiscriminantAnalysis()
                  parameters={}
                  
            # 최적 파라미터 찾기 
            self.grid_dt  = GridSearchCV(dt, param_grid = parameters, cv=5, refit =True, n_jobs=-1)
            self.grid_dt.fit(self.X_train, self.y_train)
            
            print(f"모델명: {model_name}")
            print(f"학습 데이터 최적 파라미터\n: {self.grid_dt.best_params_}")
            print(f"학습 데이터 최고 정확도\n: {self.grid_dt.best_score_:.3f}")

            #지도학습 알고리즘 
            estimator = self.grid_dt.best_estimator_
            y_pred = pd.DataFrame(estimator.predict(self.X_test))
            y_pred_probability = pd.DataFrame(estimator.predict_proba(self.X_test))
            # print(f"예측 정확도\n: {accuracy_score(self.y_test, y_pred):.3f}")

            prediction = pd.concat([y_pred_probability, y_pred], axis = 1)

            prediction.columns = ["Negative(0)", "Positive(1)", "y_pred"]

            print(prediction)
            self.pred_proba_1 = np.array(prediction["Positive(1)"]).reshape(-1, 1)
            self.accuracy= round(accuracy_score(self.y_test, y_pred),3)
            self.FPRs, self.TPRs, self.thresholds = roc_curve(self.y_test, self.pred_proba_1)
            
            return estimator



      # 단일 모델 임계치별 score 출력 하는 함수 *주의  self.thresholds 값 내부 리스트 수정해줘야함 
      def get_thresholds_score(self):
            global model_df

            
            self.thresholds = [0.1, 0.2 , 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

            #평가지표용 리스트생성
            precisions = []
            recalls = []
            f1_scores = []
            auc_scores=[]
            threshold_ =[]
            
            
            
            for threshold in self.thresholds:
                  binarizer = Binarizer(threshold= threshold)
                  # print(threshold)
                  # 임계점 지정하여 Binariazer() 객체 생성
                  pred_proba = binarizer.fit_transform(self.pred_proba_1)
                  # print(pred_proba)
                  # 임계점을 기준으로 데이터 변환

                  precision = precision_score(self.y_test, pred_proba)
                  recall = recall_score(self.y_test, pred_proba)
                  f1score = f1_score(self.y_test, pred_proba)
                  auc_score=roc_auc_score(self.y_test, self.pred_proba_1)

                  precisions.append(precision)
                  recalls.append(recall)
                  f1_scores.append(f1score)
                  auc_scores.append(auc_score)
                  threshold_.append(threshold)
                  

            results = pd.DataFrame(data = {"thresholds":threshold_, "models":self.model_name,"Accuracy":self.accuracy,"Precision": precisions,
                                          "Recall": recalls,
                                          "F1 score": f1_scores,"Auc_Score":auc_scores}, index = self.thresholds)                 

            print(f'단일 모델 results : {results}')      
            return results




      #모든 모델 성능 비교 함수 
      def get_model_socre(self,input_list):
            global model_df 

            mod = Model_Optimization()
            for i in tqdm(input_list):
                  mod.grid(model_name=i)
                  results_df = mod.get_thresholds_score()

                  #평가지표 데이터 프레임화 
                  print(results_df)
                  #기준 평가지표로 정렬
                  results_df.sort_values("F1 score", ascending=False, inplace=True)
                  new_model_df= results_df.iloc[:1]
                  model_df = pd.concat([model_df,new_model_df])
                  
            return model_df

In [16]:
list = ['LogisticRegression','GradientBoostingClassifier','XGBClassifier','KNeighborsClassifier','DecisionTreeClassifier','RandomForestClassifier','Linear SVM','RBF SVM']
model = Model_Optimization()
model.get_model_socre(list)
model_df


  0%|          | 0/8 [00:00<?, ?it/s]

모델명: LogisticRegression
학습 데이터 최적 파라미터
: {'C': 5, 'penalty': 'l2'}
학습 데이터 최고 정확도
: 0.888
      Negative(0)  Positive(1)  y_pred
0        0.986607     0.013393     0.0
1        0.958585     0.041415     0.0
2        0.987849     0.012151     0.0
3        0.956637     0.043363     0.0
4        0.976211     0.023789     0.0
...           ...          ...     ...
2168     0.450098     0.549902     1.0
2169     0.947904     0.052096     0.0
2170     0.989493     0.010507     0.0
2171     0.996955     0.003045     0.0
2172     0.972485     0.027515     0.0

[2173 rows x 3 columns]
단일 모델 results :      thresholds              models  Accuracy  Precision    Recall  F1 score  \
0.1         0.1  LogisticRegression     0.942   0.030612  0.947368  0.059308   
0.2         0.2  LogisticRegression     0.942   0.049708  0.894737  0.094183   
0.3         0.3  LogisticRegression     0.942   0.062222  0.736842  0.114754   
0.4         0.4  LogisticRegression     0.942   0.079545  0.736842  0.143590   
0.

,thresholds,models,Accuracy,Precision,Recall,F1 score,Auc_Score
0.9,0.9,LogisticRegression,0.942,0.454545,0.526316,0.487805,0.944827
0.9,0.9,GradientBoostingClassifier,0.996,1.000000,0.684211,0.812500,0.990886
0.8,0.8,XGBClassifier,0.995,1.000000,0.684211,0.812500,0.986659
0.7,0.7,KNeighborsClassifier,0.974,0.310345,0.473684,0.375000,0.802998
0.9,0.9,DecisionTreeClassifier,0.995,1.000000,0.684211,0.812500,0.976739
0.5,0.5,RandomForestClassifier,0.998,1.000000,0.736842,0.848485,0.980501
0.9,0.9,Linear SVM,0.947,0.416667,0.526316,0.465116,0.943777
0.2,0.2,RBF SVM,0.983,0.265306,0.684211,0.382353,0.834506


## 확장기 SandardScaler

In [20]:
select_std_final = ['RETA','매출액영업이익률','총자본영업이익률','매출액순이익률',
              '이자보상배율','vol매출액영업이익률','vol매출액총이익률']

expansion_std = pd.read_csv('../Data/WIN_UP/expansion_win_std_smote.csv',encoding='cp949')
test_std = pd.read_csv('../Data/WIN_UP/expansion_win_std_test.csv',encoding='cp949')

# train
X_train_std = expansion_std[select_std_final]
y_train_std = expansion_std['부도']

# test
X_test = test_std[select_std_final]
y_test = test_std.iloc[:,-1]

In [21]:
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, accuracy_score, f1_score
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression,  LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance
# from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from tqdm.notebook import tqdm
from sklearn.preprocessing import Binarizer
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import GridSearchCV

# df = pd.DataFrame(columns =['models','Accuracy','Precision', 'Recall', 'F1 score','Auc_Score'])



class Model_Optimization():

      def __init__(self):
            
            global model_df
            model_df = pd.DataFrame(columns =['thresholds','models','Accuracy','Precision', 'Recall', 'F1 score','Auc_Score']) 


      #최적 파라미터 찾는 함수
      def grid(self,model_name, X_train=X_train_std, y_train=y_train_std, X_test=X_test,y_test=y_test):
            global model_df

            self.model_name = model_name
            self.X_test = X_test
            self.y_test = y_test
            self.X_train = X_train
            self.y_train  = y_train

            if model_name == "DecisionTreeClassifier":
                  dt = DecisionTreeClassifier()
                  parameters = [{"max_depth": [3,5,7], "min_samples_split":[3,5,7]}]
                  
            elif model_name == "RandomForestClassifier":
                  dt = RandomForestClassifier()
                  parameters = [{"max_depth": [3,5,7], "min_samples_split":[3,5,7]}]
                  
            elif model_name == "LogisticRegression":
                  dt = LogisticRegression()
                  parameters = {'penalty':['l2', 'l1'],
                  'C':[0.01, 0.1, 1, 5, 10]}
                  
            elif model_name == "Linear SVM": 
                  dt = SVC(probability=True)
                  parameters = [{'kernel':['linear'], 'C':[0.001, 0.01, 0.025, 0.1, 1, 10, 100]}]
                  
            elif model_name == "RBF SVM": 
                  dt = SVC(probability=True)
                  parameters = [{'gamma':[0.001, 0.01, 0.1, 1, 10, 100],'kernel':['rbf'],'C':[0.001, 0.01, 0.025, 0.1, 1, 10, 100]}]

            elif model_name == "AdaBoostClassifier":
                  dt= AdaBoostClassifier(random_state=0)
                  parameters={'n_estimators':[5]}
                  
            elif model_name == "GradientBoostingClassifier":
                  dt=GradientBoostingClassifier(random_state=0)
                  parameters = {
                  'n_estimators' : [100, 500],
                  'learning_rate' : [0.05, 0.1]
                              }

            elif model_name == "XGBClassifier":
                  dt=XGBClassifier(random_state=0,verbose=1)
                  parameters={'n_estimators':[100, 500], 'learning_rate':[0.05, 0.1], 'max_depth':[3,4]}
                  
            # elif model_name == "LGBMClassifier":
            #       dt=LGBMClassifier(random_state=0)
            #       parameters={'n_estimators':[400,800], 'learning_rate':[0.05, 0.1] , 'max_depth':[3,4]}
            
            elif model_name == "KNeighborsClassifier":
                  dt = KNeighborsClassifier()
                  parameters = {'n_neighbors':[3]}
                  
            elif model_name == 'MLPClassifier':
                  dt = MLPClassifier(random_state=0)
                  parameters={'max_iter':[1000], 'hidden_layer_sizes':[1], 'activation':['logistic'],
                              'solver':['sgd'], 'alpha':[0.01], 'batch_size':[32],
                              'learning_rate_init':[0.1], 'max_iter':[500]}
                  
            elif model_name == 'GaussianProcessClassifier':
                  dt = GaussianProcessClassifier(random_state=0)
                  parameters={'kernel': [1.0*RBF(1.0)]}
                  
            elif model_name == 'GaussianNB':
                  dt = GaussianNB()
                  parameters={}
                  
            elif model_name =='QuadraticDiscriminantAnalysis':
                  #선형판별분석
                  dt = QuadraticDiscriminantAnalysis()
                  parameters={}
                  
            # 최적 파라미터 찾기 
            self.grid_dt  = GridSearchCV(dt, param_grid = parameters, cv=5, refit =True, n_jobs=-1)
            self.grid_dt.fit(self.X_train, self.y_train)
            
            print(f"모델명: {model_name}")
            print(f"학습 데이터 최적 파라미터\n: {self.grid_dt.best_params_}")
            print(f"학습 데이터 최고 정확도\n: {self.grid_dt.best_score_:.3f}")

            #지도학습 알고리즘 
            estimator = self.grid_dt.best_estimator_
            y_pred = pd.DataFrame(estimator.predict(self.X_test))
            y_pred_probability = pd.DataFrame(estimator.predict_proba(self.X_test))
            # print(f"예측 정확도\n: {accuracy_score(self.y_test, y_pred):.3f}")

            prediction = pd.concat([y_pred_probability, y_pred], axis = 1)

            prediction.columns = ["Negative(0)", "Positive(1)", "y_pred"]

            print(prediction)
            self.pred_proba_1 = np.array(prediction["Positive(1)"]).reshape(-1, 1)
            self.accuracy= round(accuracy_score(self.y_test, y_pred),3)
            self.FPRs, self.TPRs, self.thresholds = roc_curve(self.y_test, self.pred_proba_1)
            
            return estimator



      # 단일 모델 임계치별 score 출력 하는 함수 *주의  self.thresholds 값 내부 리스트 수정해줘야함 
      def get_thresholds_score(self):
            global model_df

            
            self.thresholds = [0.1, 0.2 , 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

            #평가지표용 리스트생성
            precisions = []
            recalls = []
            f1_scores = []
            auc_scores=[]
            threshold_ =[]
            
            
            
            for threshold in self.thresholds:
                  binarizer = Binarizer(threshold= threshold)
                  # print(threshold)
                  # 임계점 지정하여 Binariazer() 객체 생성
                  pred_proba = binarizer.fit_transform(self.pred_proba_1)
                  # print(pred_proba)
                  # 임계점을 기준으로 데이터 변환

                  precision = precision_score(self.y_test, pred_proba)
                  recall = recall_score(self.y_test, pred_proba)
                  f1score = f1_score(self.y_test, pred_proba)
                  auc_score=roc_auc_score(self.y_test, self.pred_proba_1)

                  precisions.append(precision)
                  recalls.append(recall)
                  f1_scores.append(f1score)
                  auc_scores.append(auc_score)
                  threshold_.append(threshold)
                  

            results = pd.DataFrame(data = {"thresholds":threshold_, "models":self.model_name,"Accuracy":self.accuracy,"Precision": precisions,
                                          "Recall": recalls,
                                          "F1 score": f1_scores,"Auc_Score":auc_scores}, index = self.thresholds)                 

            print(f'단일 모델 results : {results}')      
            return results




      #모든 모델 성능 비교 함수 
      def get_model_socre(self,input_list):
            global model_df 

            mod = Model_Optimization()
            for i in tqdm(input_list):
                  mod.grid(model_name=i)
                  results_df = mod.get_thresholds_score()

                  #평가지표 데이터 프레임화 
                  print(results_df)
                  #기준 평가지표로 정렬
                  results_df.sort_values("F1 score", ascending=False, inplace=True)
                  new_model_df= results_df.iloc[:1]
                  model_df = pd.concat([model_df,new_model_df])
                  
            return model_df

In [22]:
list = ['LogisticRegression','GradientBoostingClassifier','XGBClassifier','KNeighborsClassifier','DecisionTreeClassifier','RandomForestClassifier','Linear SVM','RBF SVM']
model = Model_Optimization()
model.get_model_socre(list)
model_df


  0%|          | 0/8 [00:00<?, ?it/s]

모델명: LogisticRegression
학습 데이터 최적 파라미터
: {'C': 1, 'penalty': 'l2'}
학습 데이터 최고 정확도
: 0.866
      Negative(0)  Positive(1)  y_pred
0        0.956082     0.043918     0.0
1        0.968840     0.031160     0.0
2        0.945745     0.054255     0.0
3        0.869622     0.130378     0.0
4        0.970218     0.029782     0.0
...           ...          ...     ...
2168     0.126233     0.873767     1.0
2169     0.896192     0.103808     0.0
2170     0.973636     0.026364     0.0
2171     0.997051     0.002949     0.0
2172     0.964193     0.035807     0.0

[2173 rows x 3 columns]
단일 모델 results :      thresholds              models  Accuracy  Precision    Recall  F1 score  \
0.1         0.1  LogisticRegression     0.927   0.027778  0.947368  0.053973   
0.2         0.2  LogisticRegression     0.927   0.044077  0.842105  0.083770   
0.3         0.3  LogisticRegression     0.927   0.059041  0.842105  0.110345   
0.4         0.4  LogisticRegression     0.927   0.070755  0.789474  0.129870   
0.

,thresholds,models,Accuracy,Precision,Recall,F1 score,Auc_Score
0.9,0.9,LogisticRegression,0.927,0.363636,0.421053,0.390244,0.940698
0.9,0.9,GradientBoostingClassifier,0.991,1.000000,0.684211,0.812500,0.986268
0.8,0.8,XGBClassifier,0.994,1.000000,0.684211,0.812500,0.982920
0.7,0.7,KNeighborsClassifier,0.969,0.322581,0.526316,0.400000,0.803181
0.7,0.7,DecisionTreeClassifier,0.991,0.928571,0.684211,0.787879,0.942530
0.5,0.5,RandomForestClassifier,0.998,1.000000,0.736842,0.848485,0.984509
0.9,0.9,Linear SVM,0.929,0.409091,0.473684,0.439024,0.931107
0.5,0.5,RBF SVM,0.979,0.272727,0.631579,0.380952,0.914541


In [23]:
# model_df.to_csv('../Data/result/확장기예측결과_std.csv',encoding='cp949')